In [2]:
!pip install selenium

# 필요한 패키지 불러오기

In [8]:
from selenium import webdriver
import time
import pandas as pd
from bs4 import BeautifulSoup
import datetime
from datetime import timedelta
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup as bs
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
import pickle

# 호텔별 URL을 가져오기

In [10]:
# url 함수
def make_url(Url):
    import time
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome()
    driver.get(Url)
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    click1 = driver.find_element(By.XPATH,'//*[@id="__next"]/div/header/section/div/div[1]/section[2]/div/div/div/div/div[2]/div/div[1]/span[1]/span')
    click1.click()
    time.sleep(6)
    click2 = driver.find_element(By.XPATH,'/html/body/div[12]/div/div/section/div/div[3]/div[4]/section/div[2]/div/span[1]')
    click2.click()
    click3 = driver.find_element(By.XPATH,'/html/body/div[12]/div/div/section/div/div[3]/div[4]/section/div[2]/div/span[2]')
    click3.click()
    time.sleep(2)
    click4 = driver.find_element(By.XPATH,'/html/body/div[12]/div/div/footer/div/button[2]')
    click4.click()
    time.sleep(2)
    #무한 스크롤
    
    time.sleep(1)
    
    SCROLL_PAUSE_TIME = 1.5
    
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
    # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")
        time.sleep(SCROLL_PAUSE_TIME)
        
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        if new_height == last_height:
            break
        
        last_height = new_height  

    soup = BeautifulSoup(html, 'html.parser')
    data = soup.select("section > div > div > a")
        
    for d in data:
        hotelurl.append(d.attrs['href'])
    # driver.close()
  

In [12]:
hotelurl = []
make_url("https://www.yanolja.com/hotel/r-900588?pageKey=1724956037528")

# 리뷰 수집

In [16]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome()

hotel = []
star = []
reviews = []
rev_sum_pos = []
rev_sum_neg = []
columns = ['hotel','star' ,'reviews', 'Review Summary Pos', 'Review Summary Neg','url' ]   


for url in hotelurl:
    cnt = 0
    SCROLL_PAUSE_TIME = 0.5
    time.sleep(1)
    Url = "https://www.yanolja.com"+url
    driver.get(Url)
    WebDriverWait(driver, 10)
    # try:
    # # 요소 찾기 시도
    try : 
        rev_sum_pos.append(driver.find_element(By.CLASS_NAME, 'css-4a3h28').text)
        element = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, "//div[text()='낮은 평점 요약']")))
        element.click()
        time.sleep(1)
    except NoSuchElementException:
        rev_sum_pos.append("")
    try:   
        element = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, "//div[text()='낮은 평점 요약']")))
        element.click()
        time.sleep(1)
    except TimeoutException:
        pass
    try :    
        rev_sum_neg.append(driver.find_element(By.CLASS_NAME, 'css-4a3h28').text) 
    except NoSuchElementException:
        rev_sum_neg.append("") 
    # except NoSuchElementException:
    # # 요소가 없을 경우 공백 문자열 설정
    #     rev_sum_pos = ''
    #     rev_sum_neg = ''
    
    hotel.append(driver.find_element(By.CSS_SELECTOR,'#__next > div > div > main > article > div.css-1cc3d9 > div > div.css-1e56347 > div.css-11vo59c > h1').text)
    
    # 리뷰 페이지 클릭 (Click review page)
    try:
        element2 = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CLASS_NAME,'css-xa55ms'))
        )
    except NoSuchElementException :
        element2 = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CLASS_NAME,'css-ufp1xj'))
        )
    except TimeoutException:
        element2 = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CLASS_NAME,'css-ufp1xj'))
        )
    element2.click()
    # 무한 스크롤
    import time
    time.sleep(1)
    
    # SCROLL_PAUSE_TIME = 1.5
    element = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CLASS_NAME, "css-stl2cn")))
    element.click()
    
    element = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//div[text()='평점 낮은순']")))
    element.click()
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    max = int(driver.find_element(By.CLASS_NAME,"css-12rhouj").text.replace("(","").replace(")",""))
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")
        time.sleep(SCROLL_PAUSE_TIME)
    
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if max > 1000:
            if(cnt>20):
                break
            cnt +=1
        if new_height == last_height:
            break
    
        last_height = new_height
    
    # 스크롤 끝나고 페이지 elements 가져오기
    star.append(float(driver.find_element(By.CLASS_NAME, 'css-u41ekg').text))
    soup = bs(driver.page_source, "lxml")
    
    # 특정 클래스 이름을 가진 모든 요소 찾기
    elements = soup.find_all('p')
    temp = ""
    # 데이터 추출 
    review = driver.find_elements(By.CLASS_NAME,"css-c92dc4")
    for i in review:
        temp += i.text
    reviews.append(temp)     
    
df = pd.DataFrame({"Hotel" : hotel, "Star" : star ,"Reviews" : reviews, "Review Pos" : rev_sum_pos,"Review Neg" : rev_sum_neg,"Url": hotelurl})   
df["Url"] ="https://www.yanolja.com"+df["Url"]

In [22]:
with open('gs.pickle', 'wb') as f:
    pickle.dump(df, f)

In [20]:
with open("gs.pickle", 'rb') as f:
	data = pickle.load(f)

,Hotel,Star,Reviews,Review Pos,Review Neg,Url
0,경주 유온,4.8,장점: 인스타 감성 / 석식 맛있음\n단점: 시내와 떨어져 있어 불편 / 객실이 너...,친절한 서비스와 다양한 편의 시설이 특징입니다. 체크인 과정에서 웰컴 드링크와 포토...,,https://www.yanolja.com/hotel/10054988
1,창원 하운드호텔 진해 용원,4.8,출장이 취소되는 바람에 이용할 수 없었습니다. 취소가 되지 않아 그냥 날렸네요.남친...,"전반적으로 청결하고 쾌적한 환경을 제공하며, 다양한 무료 조식이 인상적입니다. 침대...",,https://www.yanolja.com/hotel/3012173
2,호텔팰리스 경주,4.6,"오픈한지 얼마 안된 곳이라서 그런지 객실은 깨끗,깔끔합니다. 조식도 기본적인 토스트...","황리단길과 가까워 위치가 편리하며, 주변에 맛집과 야시장이 있어 관광하기 좋습니다....","청결도와 서비스에 대한 긍정적인 평가가 많으나, 일부 후기는 화장실의 악취와 좁은 ...",https://www.yanolja.com/hotel/10047293
3,대구 JB 관광호텔,4.7,새벽까지 소리지르고 뛰어다니는 진상 단체손님 제어를 못함. 그런 단체를 받으면 적어...,"청결과 편안함이 강조되는 곳으로, 주차 공간이 넓고 주변 편의시설이 잘 갖춰져 있어...",출장이나 여행을 위한 합리적인 선택지로 평가됩니다. 저렴한 가격에 조식이 맛있고 깨...,https://www.yanolja.com/hotel/1000112703
4,김해 아세라빌라드아쿠아,4.9,"방하나에 화장실?\n거실이없어서 답답한느낌이에요,\n아침에일어나니 방온도 28도.....","청결과 편안함이 강조된 곳으로, 객실 및 욕실이 매우 깔끔하게 관리되고 있습니다. ...",,https://www.yanolja.com/hotel/10051156
5,크라운호텔 창원,4.7,서비스 최악벼어어얼로예매취소정책이 아쉽습니다괜찮았어요너무비싸요잘쉬다갑니다...청소가...,"청결하고 조용한 환경을 제공하며, 편안한 온돌방과 다양한 OTT 서비스가 특징입니다...",,https://www.yanolja.com/hotel/3001551
6,경주 코오롱 호텔,3.8,너무 오래되서 그런지 노후된 객실안과 화장실을 비롯하여 객실이 너무 좁아요..또 오...,"숙소는 전반적으로 쾌적하고 청결한 환경을 제공하며, 가족 단위 여행객들에게 적합한 ...","전반적으로 노후화된 시설과 청결 상태에 대한 불만이 많이 제기되었으며, 특히 방음이...",https://www.yanolja.com/hotel/3000920
7,씨엘미니호텔,5.0,친절하시고 위치도 시내와 가까워 편리성이 좋았습니다\n다음에 방문하면 재 방문 좋습...,"시내와 가까운 위치에 있어 편리하며, 접근성이 뛰어난 점이 특징입니다. 청결도가 높...",,https://www.yanolja.com/hotel/10056854
8,경주 공간 호텔,4.6,단점은 방이 작다는거\n조식도 간편조식이지만 웬만한건 다있고\n아이스아메리카노 언제...,사장님의 친절함과 깨끗한 시설로 호감을 주는 숙소입니다. 간단하면서 맛있는 조식과 ...,,https://www.yanolja.com/hotel/10058136
9,창원 돈 호텔,4.7,출장와서 잘쉬고 다음날 출근했습니다객실 미니 세탁기 건조기 스타일러까지 없는게 없네...,직원들의 친절한 서비스와 깔끔한 숙소 상태로 높은 만족도를 주고 있습니다. 조식은 ...,,https://www.yanolja.com/hotel/10042129
